<h2> DATA 319: Model-based and Data-based Methods for Data Analytics. Summer 2024 </h2>
<h3> Problem Set 3 </h3>
<h3> Team <i> (3) </i></h3>
<h3> Type students' names <i> (only those who contributed to the group work)</i> here : Kikzely Avalos, James Mello, Benjamin Prior, John Salmon, Harpreet Tiwana</h3>

#### 1. In your own words, explain the difference between two tests: ANOVA and MANOVA. How MANOVA is different from a multi-factor ANOVA?
On a base level, ANOVA and MANOVA are different in the fact that MANOVA uses more than one dependent variables. Hence the M in MANOVA standing for Multivariate. ANOVA on the other hand is univariate, meaning it only uses one dependent variable. MANOVA uses multiple variables to compare multiple sample means. With multi factor ANOVA, you still use one dependent variable, while with MANOVA you can use multiple.

#### 2. For the following two variables

\begin{bmatrix} X & Y\\ 2 & 2\\ 0 & 0\\ -1 & 3 \\ 0 & 1\\ 0 & 1\\ 0 & 1\\ 1 & -1\\ 1 & 0 \end{bmatrix}

##### (a) Perform a hypothesis test to check whether the population means of the two variables are the same at the α = .05 significance level (assume unequal variances).
Null hypothesis (H0): suggests that the population means are equal.
Alternative Hypothesis (H1): population means are not equal.

A two-sample t-test is performed down below to compare the population means.
Based on the results, the p-value is greater than the significance level α=0.05 and therefore we fail to reject the null hypothesis. There is no difference in the population means.

##### (b) Report the value of your test statistic and the critical value of the test. Report the p-value of the test and your test decision.\
The t-test value is -0.9142 and the critical value is 2.14, and p-value is 0.377. Based on these results, we fail to reject the null hypothesis.

In [ ]:
import numpy as np
from scipy import stats

X = np.array([2, 0, -1, 0, 0, 0, 1, 1])
Y = np.array([2, 0, 3, 1, 1, 1, -1, 0])

x_mean = np.mean(X)
y_mean = np.mean(Y)
x_std = np.std(X, ddof=1)
y_std = np.std(Y, ddof=1)

size_x = len(X)
size_y = len(Y)

#assuming unequal variances
t_stat, p_value = stats.ttest_ind(X, Y, equal_var=False)
print("T-statistic:", t_stat)
print("P-value:", p_value)

df = size_x + size_y - 2
alpha = 0.05
critical = stats.t.ppf(1 - alpha/2, df)
print("Critical value:", critical)

test = abs(t_stat) > critical
print("Reject null hypothesis: ", test)

#### 3. This problem references the `baseball.csv` dataset, which has six variables (At Bats, Walks, Strikeouts, Hits, Home Runs) for 25 players from MLB 2021 season.

##### (a) Report the sample means for each of these six variables.

##### (b) Perform a hypothesis test against $\mu = (570, 60, 130, 160, 30, 90)$ representing the prior year full-league averages at the α = .05 significance level. Report the value of your test statistic and the critical value of the test. Report the p-value of the test and your test decision.

##### (c) Compute $T^2$ simultaneous confidence intervals to give coverage at the 95% confidence level.

##### (d) Compute simultaneous confidence intervals using Bonferroni correction to give coverage at the 95% confidence level.

##### (e) Use your intervals from parts (c) and (d) to determine which component(s) are causing the difference identified in your hypothesis test (if any).
Each component’s means fall within the ranges of the T^2 simultaneous confidence intervals at the 95% confidence level and Bonferroni correction intervals at the 95% confidence level. For instance, At_Bats has a mean of 571.24 and is in the range of T^2 confidence intervals at 529.79, 612.68 and Bonferroni correction interval at 544.54, 597.93. This means that each component such as At_Bats, Walks, Strikeouts, Hits, Home_Runs, and Runs_Scored are not causing a difference.

In [ ]:
import importlib

for module_name in ["numpy", "scipy", "pandas", "matplotlib", "mpl_toolkits", "hotelling"]:
    try:
        importlib.import_module(module_name)
    except ModuleNotFoundError:
        print(f"Installing module {module_name}")
        %pip install $module_name

import numpy as np
import pandas as pd
from scipy import stats
from hotelling.stats import hotelling_t2

baseball = pd.read_csv("baseball.csv")
xbar = np.mean(baseball, 0)
print("\n******************************\n")
print(f"Means of all variables:\n{xbar}")
print("\n******************************\n")

(t2, f_val, p_val, s) = hotelling_t2(baseball, np.array([570, 60, 130, 160, 30, 90]))

alpha = 0.05
n = len(baseball)
p = len(baseball.columns.tolist())
critical = ((p*(n-1))/(n-p)) * stats.f.ppf(1 - alpha,p,n-p)

print(f"The critical value is {critical_val} while the test statistic is {t2}")
print(f"It is {str(abs(t2) > critical).lower()} that we should reject the null hypothesis")
print("\n******************************\n")

S = np.cov(baseball.T)
simultaneous_intervals = []

for i in range(p):
    simultaneous_intervals.append([xbar.iloc[i] - np.sqrt(critical * S[i,i]/n), xbar.iloc[i] + np.sqrt(critical * S[i,i]/n) ])

print("T^2 Simultaneous confidence intervals at the 95% confidence level:")
for entry in zip(baseball.columns, simultaneous_intervals):
    print(f"  * {entry[0]} - low {entry[1][0]}, high {entry[1][1]}")
print("\n******************************\n")

bonf_alpha = alpha / p
bonferroni_correction = []

for i in range(p):
    bonferroni_correction.append(stats.ttest_1samp(baseball.iloc[:,i],xbar.iloc[i]).confidence_interval(confidence_level=1-bonf_alpha))

print("Bonferroni correction intervals at the 95% confidence level:")
for entry in zip(baseball.columns, bonferroni_correction):
    print(f"  * {entry[0]} - low {entry[1][0]}, high {entry[1][1]}")
print("\n******************************\n")

#### 4. Consider the following multivariate dataset

#### $X = \begin{bmatrix}2&2&3\\0&0&2\\-1&3&2\\0&1&1\\0&1&5\\0&1&3\\1&-1&3\\1&0&5\end{bmatrix}$

##### (a) Perform a hypothesis test with the null hypothesis $H_0 : (\mu_1 - \mu_2, \mu_1 - \mu_3) = (0, 0)$ at the α = .05 significance level. Obtain the differences using the contrast matrix. Report the value of your test statistic and the critical value of the test. Report the p-value of the test and your test decision.

For α = 0.05 significance level, the test statistic is 27.8127 which is greater than the critical value at 11.5722. The p-value is 0.00906 which is less than α = 0.05. This means that in the test decision we reject the null hypothesis.

##### (b) Will your test decision change at α = .01? At α = .001?

For α = 0.01 significance level, the test statistic is 27.8127 which is slightly greater than the critical value at 26.5478. Also, since the p-value is less than α = 0.01, we reject the null hypothesis. For α = 0.001 significance level, the test statistic is actually less than the critical value which is 74.2446 and the p-value is 0.00906 which is greater than α = 0.001. This means that we fail to reject the null hypothesis. Overall, the test decision does change at α = 0.001 but is the same for α = 0.05 and α = 0.01.


In [ ]:
# part a:

import numpy as np
from scipy import stats

# making table into a matrix
X = np.array([[2, 2, 3],
[0, 0, 2],
[-1, 3, 2],
[0, 1, 1],
[0, 1, 5],
[0, 1, 3],
[1, -1, 3],
[1, 0, 5]])

# getting the sample means
sample_means = X.mean(axis=0)

# using contrast matrix to get the differences
contrast = np.array([[1, -1, 0],
              [1, 0, -1]])

mean_diffs = np.dot(contrast, sample_means)

# number of observations and variables
n, p = X.shape

# getting the covariance matrix
covariance = np.cov(X, rowvar=False)

# test statistic and degrees of freedom
inv = np.linalg.inv(np.dot(np.dot(contrast, covariance), contrast.T))
test_statistic = n * np.dot(np.dot(mean_diffs.T, inv), mean_diffs)

df1 = contrast.shape[0]
df2 = n - p

# critical value and p-value for α = .05 significance level
alpha = 0.05

critical_value = df1 * stats.f.ppf(1 - alpha, df1, df2)
p_value = 1 - stats.f.cdf(test_statistic / df1, df1, df2)

results_for_05 = (test_statistic, critical_value, p_value)

print("α = .05:", results_for_05)

# part b: checking different significance levels for α = 0.01 and α = 0.001
# critical value and p-value
alpha1 = 0.01
critical_value1 = df1 * stats.f.ppf(1 - alpha1, df1, df2)

alpha2 = 0.001
critical_value2 = df1 * stats.f.ppf(1 - alpha2, df1, df2)

results_for_01 = (critical_value1, p_value < alpha1)
results_for_001 = (critical_value2, p_value < alpha2)

print("α = .01:", results_for_01)
print("α = .001:", results_for_001)

α = .05: (27.812709030100336, 11.572270086699929, 0.009063997819291791)
α = .01: (26.54786722400965, True)
α = .001: (74.24465962305564, False)


#### 5. This problem references the `anova.csv` dataset, which has three numerical columns: $X_1$, $X_2$, and $X_3$ and a fourth categorical column that separates the observations into three classes: "A", "B", and "C".

##### (a) Evaluate whether the data $(X_1, X_2, X_3)$ arises from a multivariate normal distribution using visual analysis tools. Construct univariate histograms, univariate probability plots, and pairwise scatterplots.
On examination it looks as though the $x_2$ and $x_3$ plots are normally distributed as there appears to be a bell curve in the histogram plot and the probability plots have a well fitted diagonal line. the $x_1$ column does not appear to be normal because its histogram is scattered and and probability plot does not appear linear. It will have to be adjusted.

##### (b) If you decided in part (a) that any of the columns were not normal, perform a suitable transformation to alleviate this problem.
See Code Below
##### (c) Using your (potentially) transformed dataset from part (b) how would you test if the population means across all three groups are equal? State your null hypothesis.
Using ANOVA or ANalysis Of VAriance we can test the differences between the means of the populations or columns. The null hypothesis is that $H_0 = \mu_1 = \mu_2 = \mu_3$ or the population means are all equal. The alternative hypothesis is that one or more of the means does not equal the others.
##### (d) Perform a hypothesis test at the α = .05 significance level. What is the test statistic used for this test? Report the p-value of the test and your test decision.
The test statistic for ANOVA is the F value which tells us the ratio of variance between groups to the variance of groups. It should be close to 1 if our null hypothesis is true because the variance would be roughly equal. In this case it was 42 but our P value was very very small so we reject the null hypothesis and conclude that the means are not the same.

In [ ]:
#%pip install -q pandas matplotlib scipy numpy seaborn
import pandas as pd
import matplotlib.pyplot as plt
from scipy import stats
import numpy as np
import seaborn as sns

#create DF from csv
anova_pd = pd.read_csv('anova.csv')
anova_pd.drop('Type', axis = 1, inplace = True) #drop the Type column as its not numeric
anova_pd.drop(anova_pd.columns[0], axis = 1, inplace = True) #drop the first index column



#A
#Histogram
for column in anova_pd:
    plt.figure()
    anova_pd[column].hist()
    plt.title(column + ' Histogram')

#Probability plot
for column in anova_pd:
    plt.figure()
    stats.probplot(anova_pd[column], plot = plt)
    plt.title(column + ' Probability Plot')

#Pairwise scatter plot
sns.pairplot(anova_pd)
plt.show()



In [ ]:
#B
#Log Transform X_1 Column
anova_pd['X1'] = anova_pd['X1'].apply(lambda x: (.5*np.log(x/(1-x))))
plt.figure()
anova_pd['X1'].hist()
plt.title('X1 Histogram After Transform')
plt.figure()
stats.probplot(anova_pd['X1'], plot = plt)
plt.title('X1 Probability Plot After Transform')
plt.show()

In [ ]:
#D
#ANOVA
alpha = 0.05
f_value, p_value = stats.f_oneway(anova_pd['X1'], anova_pd['X2'], anova_pd['X3'])
print('F Value:', f_value)
print('P Value:', p_value)

if p_value < alpha:
    print('Reject Null Hypothesis')
else:
    print('Fail to Reject Null Hypothesis')